# 금연구역_2차_주소전처리 시도

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# import tensorflow as tf

import requests
import json


In [2]:
# PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data'
PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'

df_fail = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_실패(11115).csv', encoding ='cp949')
df_save = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_성공(32752).csv', encoding ='cp949')

## 실패한 주소 체크

전처리를 시도한 kakao 주소가 아닌 기존의 원주소로 다시 api 호출을 시도해봄

In [3]:
df_fail.shape, df_save.shape

((11115, 10), (32752, 10))

In [4]:
df_fail.head()

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,대전 보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 대전 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,대전 태평동 삼부@ 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 대전 태평동 삼부@ 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,대전 선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 대전 선화로57번길 8,NaN,NaN,NaN


### kakao api 세팅

In [6]:
# # rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

rest_api_key = "616734aa8214dbf958cfaebd6bd41aec"

In [7]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [8]:
kakao = KakaoLocalAPI(rest_api_key)

In [10]:
df_fail2 = df_fail.copy()

In [11]:
add_list = df_fail2['번지주소']
add_list[0]

'대전광역시 중구 계백로1571번길 46 , 2층 (유천동)'

In [13]:
kakao.search_address(add_list[0])

{'documents': [{'address': {'address_name': '대전 중구 중촌동 116-8',
    'b_code': '3014010400',
    'h_code': '3014056000',
    'main_address_no': '116',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '중촌동',
    'region_3depth_name': '중촌동',
    'sub_address_no': '8',
    'x': '127.414436051345',
    'y': '36.3387070586325'},
   'address_name': '대전 중구 목중로26번길 83',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '대전 중구 목중로26번길 83',
    'building_name': '',
    'main_building_no': '83',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.414436051345',
    'y': '36.3387070586325',
    'zone_no': '34804'},
   'x': '127.414436051345',
   'y': '36.3387070586325'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [14]:
# 호출에 실패한 리스트만 번지주소로 다시 돌려봄
for i in tqdm(range(len(add_list))):
    try :
        df_fail2['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
        df_fail2['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_fail2['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass

print(df_fail2.isnull().sum())

  0%|          | 0/11115 [00:00<?, ?it/s]C:\Users\Helpe\AppData\Local\Temp\ipykernel_5288\3131939199.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fail2['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
C:\Users\Helpe\AppData\Local\Temp\ipykernel_5288\3131939199.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fail2['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
C:\Users\Helpe\AppData\Local\Temp\ipykernel_5288\3131939199.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

구               0
구분              0
시설명             0
주소              0
번지주소            0
세부주소        10610
kakao 주소        0
동            6523
위도           6523
경도           6523
dtype: int64


번지주소로 api 호출 후 4592개 성공, 6523개 실패

### 2차 api 호출 성공/실패 데이터프레임 저장

In [ ]:
df_fail2 = df_fail[df_fail['위도'].isnull()]
df_save2 = df_fail[~df_fail['위도'].isnull()]

print(df_fail2.shape)
print(df_save2.shape)

(11115, 10)
(0, 10)


In [ ]:
# 병합
df_save2= pd.concat([df_save3, df_save], 
                    axis=0,
                    join='outer'
                    )


df_save2.reset_index(drop=True)